In [18]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv
import os

load_dotenv(override=True)

# ALL_IN_ONE_WORKER = True
ALL_IN_ONE_WORKER = False

<h3>Define a Message Class</h3>

In [19]:
@dataclass
class Message:
    content:str

<h3> Define a host for distributed runtime</h3>

In [20]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

<h3>Create a Tool</h3>

In [21]:
serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name="internet_search",func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [22]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [23]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash"
            },
        )
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash"
            },
        )
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=os.getenv('GEMINI_ACCESS_KEY'),
            model_info={
                "vision": False,
                "function_calling": True,
                "json_output": False,  # or True if you expect structured responses
                "family": "gemini-2.5-flash"
            },
        )
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [24]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

/Users/himanshu.r.singh/Desktop/AgenticAI/agenticAI/lib/python3.13/site-packages/autogen_ext/models/openai/_openai_client.py:439: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [25]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [26]:
display(Markdown(response.content))

## Pros of AutoGen:
AutoGen offers several compelling advantages for new AI Agent projects, including:

*   **Multi-Agent Collaboration:** It excels at orchestrating dynamic conversations and collaborations between multiple AI agents, allowing them to work together like a team to solve complex tasks.
*   **Scalability and Modularity:** The framework is designed for scalability and modularity, making it suitable for both simple and complex multi-agent scenarios and facilitating the creation of customizable systems.
*   **Ease of Use & Development Acceleration:** AutoGen provides an easy-to-use and flexible framework that accelerates the development and research of agentic AI. It simplifies the creation of collaborative, task-oriented agents.
*   **Customization:** It offers high customization, allowing users to define specific roles, capabilities, and communication protocols for each agent.
*   **Integrated Observability & Debugging:** AutoGen includes integrated tools for observing and debugging agent workflows, simplifying monitoring and control.
*   **Human-in-the-Loop Capabilities:** It supports human-in-the-loop workflows, ensuring that human oversight and intervention can be integrated into AI-powered processes.
*   **LLM Optimization:** AutoGen is also noted for its capabilities in LLM optimization within multi-agent setups.

These features make AutoGen a strong candidate for projects requiring sophisticated, collaborative, and scalable AI agent solutions.

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for a new AI Agent project:

*   **Still Under Development:** AutoGen is relatively new and actively being developed. This can mean less stability, fewer mature features, and potential breaking changes in future updates.
*   **Steep Learning Curve/Complexity:** While powerful, AutoGen can be complex to set up and configure, especially for advanced multi-agent workflows. There's a trade-off between customization and ease of use, and achieving fine-grained control often requires significant effort.
*   **Limitations in Creativity (for some tasks):** While good at structured content generation and task execution, AutoGen might lack the "creativity" or nuanced understanding required for highly open-ended or subjective tasks where human-like improvisation is needed. It operates based on defined agents and their roles.
*   **Suitability for Customer-Facing Applications:** There are concerns about its direct suitability for highly sensitive or critical customer-facing applications without significant additional layers of validation and control, due to the nature of autonomous agent interactions.
*   **Resource Intensiveness:** Running multiple agents, especially for complex tasks, can be resource-intensive in terms of computational power and API calls.



## Decision:

Based purely on the research provided, I recommend **using AutoGen** for a new AI Agent project.

**Rationale:**
AutoGen's strengths, particularly its multi-agent collaboration, scalability, modularity, and development acceleration features, are highly compelling for a *new* AI agent project. It offers a robust framework for building complex, collaborative systems with good customization and integrated debugging. While the "still under development" and "steep learning curve" cons are valid concerns, the benefits of accelerating development and enabling sophisticated multi-agent interactions appear to outweigh these risks for a new project, especially one that can tolerate potential changes or invest in the initial learning. Concerns about creativity limitations or direct suitability for highly sensitive customer-facing applications are important but can be managed or considered in later project phases, rather than being immediate blockers for *starting* a new agent project.

TERMINATE

In [16]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [17]:
await host.stop()